In [4]:
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import MultiStepLR
import torch.nn.functional as F
from torch import nn
import torch
import os
import gpytorch
import math
import tqdm


In [86]:
#GaussianProcessLayer
class GaussianProcessLayer(gpytorch.models.ApproximateGP):
    def __init__(self, num_dim, grid_bounds=(-10., 10.), grid_size=64):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            num_inducing_points=grid_size, batch_shape=torch.Size([num_dim])
        )

        variational_strategy = gpytorch.variational.GridInterpolationVariationalStrategy(
            self, grid_size=grid_size, grid_bounds=[grid_bounds],
            variational_distribution=variational_distribution,
        )

        super(GaussianProcessLayer, self).__init__(variational_strategy)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size([num_dim])),
            batch_shape=torch.Size([num_dim])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        latent_pred = gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
        return latent_pred


In [87]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#data is time series data
feature_dim = 54
input_len = 128
hidden_dim = 64
kernel_size = 63

features_extractor = nn.Sequential()
features_extractor.append(nn.Conv1d(feature_dim, hidden_dim, kernel_size, padding = kernel_size//2))
features_extractor.append(nn.ReLU())
features_extractor.append(nn.Conv1d(hidden_dim, hidden_dim, kernel_size, padding = kernel_size//2))

Sequential(
  (0): Conv1d(54, 64, kernel_size=(63,), stride=(1,), padding=(31,))
  (1): ReLU()
  (2): Conv1d(64, 64, kernel_size=(63,), stride=(1,), padding=(31,))
)

In [88]:
class DKLModel(gpytorch.Module):
    def __init__(self, feature_extractor, num_dim, grid_bounds=(-10., 10.)):
        super(DKLModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.gp_layer = GaussianProcessLayer(num_dim=num_dim, grid_bounds=grid_bounds)
        self.grid_bounds = grid_bounds
        self.num_dim = num_dim

        # This module will scale the NN features so that they're nice values
        self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(self.grid_bounds[0], self.grid_bounds[1])

    def forward(self, x):
        x = x.transpose(-1, -2)
        features = self.feature_extractor(x)
        features = self.scale_to_bounds(features)
        # This next line makes it so that we learn a GP for each feature
        print(features.shape)
        features = features.transpose(-1, -2)
        res = self.gp_layer(features)
        return res

model = DKLModel(features_extractor, num_dim = hidden_dim)
likelihood = gpytorch.likelihoods.GaussianLikelihood()


# If you run this example without CUDA, I hope you like waiting!
if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

In [89]:
num_samples = 1000
#test the model
X = torch.randn(num_samples, 54)
Y = torch.randn(num_samples, 1)

batch_size = 32
day = torch.randint(0, 30, (num_samples, 1))
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, X, y, day, input_len=128):
        #the input data is a 1d array, indicate the minute of the day
        self.X = X
        self.y = y
        self.day = day
        self.input_len = input_len

    def __getitem__(self, index):
        #output previous self.input_len minutes data and target value
        #if the there is no enough data in the same day, pad with 0
        d = self.day[index]
        start = index - self.input_len
        if start < 0:
            start = 0
        if self.day[start] != d:
            while self.day[start] != d:
                start += 1
            #pad with 0 before start
        if index - start < self.input_len:
            x = torch.zeros(self.input_len, self.X.shape[1])
            x[self.input_len - index + start: self.input_len] = self.X[start: index].clone()
        else:
            x = self.X[start: index]
            
        y = self.y[index].clone()
        if x.shape != (self.input_len, self.X.shape[1]):
            print(x.shape, index, start)
        return x, y
    def __len__(self):
        return len(self.X)
    
mydataset = MyDataset(X, Y, day, input_len=128)
train_index = range(0, int(num_samples * 0.8))
test_index = range(0, int(num_samples * 0.8))
#test_index = range(int(num_samples * 0.8), num_samples)

train_dataset = torch.utils.data.Subset(mydataset, train_index)
test_dataset = torch.utils.data.Subset(mydataset, test_index)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [90]:
n_epoch = 100
batch_size = 100
lr = 0.1
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-4},
    {'params': model.gp_layer.hyperparameters(), 'lr': lr * 0.01},
    {'params': model.gp_layer.variational_parameters()},
    {'params': likelihood.parameters()},
], lr= lr)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones= [n_epoch//2, n_epoch//4*3], gamma= 0.1)
mll = gpytorch.mlls.VariationalELBO(likelihood, model.gp_layer, num_data= len(train_dataset))


In [91]:
model.train()
likelihood.train()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for i in range(n_epoch):
    optimizer.zero_grad()
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        loss = -likelihood(output, y)
        loss.backward()
        optimizer.step()

torch.Size([32, 64, 128])


AssertionError: 

In [ ]:
features_extractor.forward(x)

RuntimeError: Given groups=1, weight of size [64, 54, 63], expected input[32, 128, 54] to have 54 channels, but got 128 channels instead

In [44]:
x.shape

torch.Size([128, 128, 296])